U = 1.50V
I = 0.93A

U = 2.00V
I = 0.97A

U = 2.50V
I = 0.95A

U = 3.00V
I = 0.95A

U = 4.00V
I = 1.00A

Odtud a dál, I kolísá mezi 1.06 - 1.09 a 1.12 - 1.16

U = 5.00V
I = 1.08A

U = 6.00V
I = 1.14A

U = 7.00V
I = 1.10A

U = 8.00V
I = 1.12A

